In [3]:
# Import dependencies
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [6]:
# Get the data from the csv file
city_data_df = pd.read_csv("weather_data/WeatherPy_challenge.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain Inches (last 3 hrs),Snow Inches (last 3 hrs)
0,0,Alta Floresta,BR,2019-11-10 05:50:13,-9.87,-56.08,70.41,99,100,2.68,overcast clouds,0.0,0.0
1,1,Iwanai,JP,2019-11-10 05:50:13,42.97,140.51,40.10,62,47,10.31,scattered clouds,0.0,0.0
2,2,Arvika,SE,2019-11-10 05:50:14,59.65,12.59,28.00,100,69,10.29,broken clouds,0.0,0.0
3,3,Hamilton,CA,2019-11-10 05:50:14,43.26,-79.87,39.20,74,75,8.05,broken clouds,0.0,0.0
4,4,Avarua,CK,2019-11-10 05:48:33,-21.21,-159.78,73.40,83,0,6.93,light rain,0.0,0.0


In [ ]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [ ]:
# Ask the customer if they prefer rain or snow.
desired_rain = input("Do you want it to be raining? (yes/no) ")
desired_snow = input("Do you want it to be snowing? (yes/no) ")

In [ ]:
# Filter the dataset to find the cities that fit the temp criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

In [ ]:
# Filter the dataset based on rain preferences.
if desired_rain == 'yes':
    rain_preferred_cities_df = preferred_cities_df.loc[(preferred_cities_df["Rain Inches (last 3 hrs)"] > 0)]
else:
    rain_preferred_cities_df = preferred_cities_df.loc[(preferred_cities_df["Rain Inches (last 3 hrs)"] == 0)]
# Filter the dataset based on snow preferences.
if desired_snow == 'yes':
    ideal_preferred_cities_df = rain_preferred_cities_df.loc[(rain_preferred_cities_df["Snow Inches (last 3 hrs)"] > 0)]
else:
    ideal_preferred_cities_df = rain_preferred_cities_df.loc[(rain_preferred_cities_df["Snow Inches (last 3 hrs)"] == 0)]

In [ ]:
len(ideal_preferred_cities_df)

In [ ]:
ideal_preferred_cities_df

In [ ]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and current weather description.
hotel_df = ideal_preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

In [ ]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [ ]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to the params dictionary as values to the location key.
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

In [ ]:
hotel_df

In [ ]:
# Add info boxes to markers.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [ ]:
# Add a marker layer map for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

In [ ]:
# Create the output file (CSV).
output_data_file = "weather_data/WeatherPy_vacation.csv"
# Export the City_Data into a CSV.
hotel_df.to_csv(output_data_file, index_label="City_ID")